In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('RealOrFakeNews.csv')
df.head(10)

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1
6,6,DR BEN CARSON TARGETED BY THE IRS: “I never ha...,DR. BEN CARSON TELLS THE STORY OF WHAT HAPPENE...,1
7,7,HOUSE INTEL CHAIR On Trump-Russia Fake Story: ...,,1
8,8,Sports Bar Owner Bans NFL Games…Will Show Only...,"The owner of the Ringling Bar, located south o...",1
9,9,Latest Pipeline Leak Underscores Dangers Of Da...,"FILE – In this Sept. 15, 2005 file photo, the ...",1


In [2]:
df.shape

(72134, 4)

In [3]:
df = df.sample(50000)
df = df.reset_index(drop=True)

In [4]:
df.label.value_counts()

1    25702
0    24298
Name: label, dtype: int64

In [5]:
df.drop(['text', 'Unnamed: 0'], axis=1, inplace=True)

In [6]:
df.tail()

,title,label
49995,Kid Rock hints online he will run for U.S. Senate,0
49996,BREAKING: HOUSE Votes “Yes” On American Securi...,1
49997,Russian analysis shows western-backed terroris...,1
49998,RACIST LIBERAL REPORTER Arrested In Connection...,1
49999,Area Dad Informs Busboy He’s Ready To Order - ...,1


In [14]:
import spacy
nlp = spacy.load("C:/Python311/Lib/site-packages/en_core_web_lg/en_core_web_lg-3.5.0")

In [17]:
df['vector'] = df['title'].apply(lambda text: nlp(str(text)).vector)

In [20]:
df

,title,label,vector
0,Bill O’Reilly Sets Return with ’No Spin News’ ...,0,"[0.17620276, -1.4139764, 3.0219228, -0.9061939..."
1,How Clinton Campaign Gamed Super PAC Regulations,1,"[-0.5956214, -0.46154287, -0.86808974, 1.77138..."
2,"Opposed from the start, the rocky history of N...",0,"[-2.636486, -1.1986736, 0.298665, 0.6471771, 7..."
3,North Korea calls for a halt to 'brutal' sanct...,0,"[-0.25380728, 2.5123365, 0.19647375, 1.0413374..."
4,"Trump to speak with leaders of Russia, Germany...",0,"[-2.7611842, 0.034197945, -0.8260793, 2.22033,..."
...,...,...,...
49995,Kid Rock hints online he will run for U.S. Senate,0,"[-1.415254, 2.280379, -1.383658, 0.789571, 2.3..."
49996,BREAKING: HOUSE Votes “Yes” On American Securi...,1,"[0.31557387, -0.2993775, -0.08687037, -0.05879..."
49997,Russian analysis shows western-backed terroris...,1,"[-2.5575242, -0.32569584, -2.114174, 2.1205568..."
49998,RACIST LIBERAL REPORTER Arrested In Connection...,1,"[0.30915013, -2.1061316, 1.8317485, -0.0374743..."


In [22]:
df.vector[0].shape

(300,)

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.vector.values, df.label, test_size=0.2, random_state=100)

In [26]:
print(X_train.shape)
print(X_test.shape)

(40000,)
(10000,)


In [28]:
## We need to change my X_train and X_test to 2d
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)
X_train_2d

array([[ 0.18795997, -0.72642744, -2.8503466 , ..., -0.5008309 ,
        -1.9389362 , -0.348796  ],
       [-1.6632813 , -2.0633893 ,  1.3748507 , ..., -2.7067983 ,
         0.12365448,  0.6933477 ],
       [-1.5525672 ,  0.23355706, -0.2804629 , ..., -3.5037427 ,
        -0.86371   ,  0.7213043 ],
       ...,
       [ 1.0712537 ,  2.0026376 ,  0.03900153, ..., -2.0514636 ,
        -3.5113702 ,  3.8850694 ],
       [ 0.9224053 ,  3.1436272 ,  1.0704844 , ..., -0.57489085,
        -1.3476293 ,  2.6314955 ],
       [-0.3168118 ,  0.09268855,  0.17032267, ..., -0.7450201 ,
        -1.2926195 ,  1.4981995 ]], dtype=float32)

In [30]:
## we will do minmax scalar to remove -ve values 
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
scaled_train = sc.fit_transform(X_train_2d)
scaled_test  = sc.transform(X_test_2d)

In [31]:

from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(scaled_train, y_train)
y_pred = mnb.predict(scaled_test)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.80      0.82      4879
           1       0.82      0.85      0.83      5121

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [42]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=10, metric='euclidean')
kn.fit(scaled_train, y_train)
y_pred = kn.predict(scaled_test)

In [43]:
# KNN really works well with low dimension/Dense representation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4879
           1       0.93      0.90      0.91      5121

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000



In [44]:
y_test[:5]

4253     1
30076    1
48047    0
1666     0
30740    0
Name: label, dtype: int64

In [45]:
y_pred[:5]

array([1, 1, 0, 0, 0], dtype=int64)

In [65]:
df.title[3]

"North Korea calls for a halt to 'brutal' sanctions"

In [109]:
text = nlp('''President Joe Biden unveiled a $2 trillion infrastructure plan that would rebuild roads, bridges and airports, invest in clean energy and create millions of jobs over the next eight years. The proposal includes investments in transportation, broadband, water systems, power grids and more''')
new_test = text.vector


In [110]:
new_t = np.stack(new_test)

In [112]:
scaled = sc.transform([new_t])
kn.predict(scaled)
if result==0:
    print("Real")
else:
    print("Fake")

Real
